# MetaPulsar Usage Example

This notebook demonstrates how to use MetaPulsar to combine pulsar timing data from multiple PTA collaborations into unified Enterprise pulsar objects.

## Overview

The workflow covers:
1. **Manual single-pulsar data preparation** - Creating data structures by hand
2. **MetaPulsar creation** - Using consistent parameter merging strategies
3. **Enterprise integration** - Working with the resulting pulsar objects
4. **Automated discovery** - Processing multiple pulsars automatically
5. **Reference PTA selection** - Different strategies for choosing reference PTAs

## Key Concepts

- **Reference PTA**: The PTA whose parameters are inherited by the MetaPulsar for merged model components
- **Consistent Strategy**: Merges compatible parameters from different PTAs where possible
- **Component Merging**: Controls which parameter types (astrometry, spindown, binary, dispersion) are merged
- **Parameter Naming**: Merged parameters have no suffix, PTA-specific parameters retain PTA suffixes


In [1]:
import loguru
import sys
from metapulsar import (
    MetaPulsarFactory,
    discover_files,
    discover_layout,
    combine_layouts,
)

# Suppress debug output for cleaner example
loguru.logger.remove()
loguru.logger.add(sys.stdout, level="WARNING")


/opt/venvs/pta/lib/python3.10/site-packages/enterprise/signals/utils.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import Requirement, resource_filename


1

## Part 1: Manual Single-Pulsar Data Preparation

For single pulsars, manually creating the data structure is often the most transparent and flexible approach. This gives you full control over which PTAs to include and their ordering. It is a bit of work though, so we will go over more convenient methods later

### Data Structure

The data structure is a dictionary where:
- **Keys**: PTA names (e.g., 'nanograv_9y', 'epta_dr2')
- **Values**: Lists of file information dictionaries
- **Reference PTA**: The first PTA in the dictionary becomes the reference PTA

### File Information Fields
- `par`: Path to .par file (pulsar parameters)
- `tim`: Path to .tim file (timing observations)  
- `timing_package`: Software used (tempo2/pint)


In [2]:
# Manually create a single-pulsar dictionary with three PTAs
# The reference PTA (first in the dictionary) will be used for parameter
# inheritance where appropriate
pulsar_data = {
    # Reference PTA - parameters from this PTA will be inherited by the MetaPulsar
    # for model components that are merged (astrometry, spindown, binary, dispersion)
    "epta_dr2": [
        {
            "par": "../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200.par",
            "tim": "../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim",
            "timing_package": "tempo2",  # Timing package used
        }
    ],
    "nanograv_9y": [
        {
            "par": "../../data/ipta-dr2/NANOGrav_9y/par/J0613-0200_NANOGrav_9yv1.gls.par",
            "tim": "../../data/ipta-dr2/NANOGrav_9y/tim/J0613-0200_NANOGrav_9yv1.tim",
            "timing_package": "pint",
        }
    ],
    "ppta_dr2": [
        {
            "par": "../../data/ipta-dr2/PPTA_dr1dr2/par/J0613-0200_dr1dr2.par",
            "tim": "../../data/ipta-dr2/PPTA_dr1dr2/tim/J0613-0200_dr1dr2.tim",
            "timing_package": "tempo2",
        }
    ],
}


### MetaPulsar Creation with Consistent Strategy

The `consistent` strategy merges parameters from different PTAs where possible, inheriting values from the reference PTA for merged model components.

### Combination Strategy Options
- `consistent`: Merge compatible parameters, inherit reference PTA values
- `composite`: Keep all parameters separate with PTA suffixes

### Mergeable Components
- `astrometry`: Position, proper motion, parallax
- `spindown`: Spin frequency and derivatives
- `binary`: Binary orbital parameters
- `dispersion`: Dispersion measure and derivatives


#### Consistent is the only ''valid'' method
When creating a `consistent` MetaPulsar, the pulsar really only has a single sky position, binary model, and other relevant Astrophysical models. It's a truly unified model. Instead, the `composite` combination method keeps all parameters separate, which is not physical. Each pulsar dataset from each PTA then has separate model parameters, which is not physical. That approach is sometimes called the ''Borg method'' or the ''Frankenstat'' method in the community.

In [3]:
# Create MetaPulsarFactory instance
factory = MetaPulsarFactory()

# Create MetaPulsar using the 'consistent' strategy
# This merges parameters from different PTAs where possible
metapulsar = factory.create_metapulsar(
    file_data=pulsar_data,
    combination_strategy="consistent",  # Merge compatible parameters
    combine_components=[
        "astrometry",
        "spindown",
        "binary",
        "dispersion",
    ],  # Components to merge
    add_dm_derivatives=True,  # Ensure DM1, DM2 are present
)

print(f"Created MetaPulsar: {metapulsar.name}")
print(f"Reference PTA: {list(pulsar_data.keys())[0]}")
print("Combination strategy: consistent")
print("Components merged: astrometry, spindown, binary, dispersion")


2025-10-09 19:43:27.296 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: ELL1.
2025-10-09 19:43:27.306 | WARNING  | pint.models.model_builder:__call__:229 - UNITS is not specified. Assuming TDB...
2025-10-09 19:43:27.475 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: BT.


/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:406: UserWarning: PINT only supports 'T2CMETHOD IAU2000B'
  warn("PINT only supports 'T2CMETHOD IAU2000B'")


2025-10-09 19:43:27.590 | WARNING  | pint.models.timing_model:validate:426 - PINT does not support 'UNITS TCB' internally. Reading this par file nevertheless because the `allow_tcb` option was given. This `TimingModel` object should not be used for anything except converting to TDB.
2025-10-09 19:43:27.592 | WARNING  | pint.models.tcb_conversion:convert_tcb_tdb:133 - Converting this timing model from TCB to TDB. Please note that the TCB to TDB conversion is only approximate and the resulting timing model should be re-fit to get reliable results.


/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:400: UserWarning: PINT does not support 'DILATEFREQ Y'
  warn("PINT does not support 'DILATEFREQ Y'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:403: UserWarning: PINT only supports 'TIMEEPH FB90'
  warn("PINT only supports 'TIMEEPH FB90'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:432: UserWarning: START cannot be unfrozen... Setting START.frozen to True
  warn("START cannot be unfrozen... Setting START.frozen to True")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:435: UserWarning: FINISH cannot be unfrozen... Setting FINISH.frozen to True
  warn("FINISH cannot be unfrozen... Setting FINISH.frozen to True")


2025-10-09 19:43:27.818 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: ELL1.
2025-10-09 19:43:27.994 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: BT.
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim


...

Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0200/tims/JBO.DFB.1400.tim
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0200/tims/JBO.DFB.1520.tim
[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0200/tims/EFF.EBPP.1360.tim
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0200/tims/EFF.EBPP.1410.tim
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0200/tims/EFF.EBPP.2639.tim
Current filename = ../../data/ipta-dr2/EPTA_v2.2/J0613-0200/J0613-0200_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0613-0

/opt/venvs/pta/lib/python3.10/site-packages/pint/observatory/clock_file.py:183: UserWarning: Data points out of range in clock file 'tai2tt_bipm2011.clk'
  warn(msg)


[preProcess.C:158] Warning: PSR J0613-0200 uses DM2+ but does not define DM_SERIES. Assume Taylor. This has behaviour has changed since June 2020!
See https://bitbucket.org/psrsoft/tempo2/issues/27/tempo2-dm-polynomial-is-not-a-taylor



Results for PSR J0613-0200


RMS pre-fit residual = 0.000 (us), RMS post-fit residual = 63.263 (us)
Fit Chisq = 0	Chisqr/nfree = 0.00/0 = nan	pre/post = 0
Number of fit parameters: 0
Number of points in fit = 0
Offset: 0 1 offset_e*sqrt(n) = 0 n = 0


PARAMETER       Pre-fit                   Post-fit                  Uncertainty   Difference   Fit
---------------------------------------------------------------------------------------------------
RAJ (rad)       1.63071752891066          1.63071752891066          0             0             Y
RAJ (hms)       06:13:43.9756763           06:13:43.9756763         0             0            
DECJ (rad)      -0.0351355399976385       -0.0351355399976385       0             0             Y
DECJ (dms)      -02

/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:400: UserWarning: PINT does not support 'DILATEFREQ Y'
  warn("PINT does not support 'DILATEFREQ Y'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:403: UserWarning: PINT only supports 'TIMEEPH FB90'
  warn("PINT only supports 'TIMEEPH FB90'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'DMASSPLANET1 0'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'DMASSPLANET2 0'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/mod

Created MetaPulsar: J0613-0200
Reference PTA: epta_dr2
Combination strategy: consistent
Components merged: astrometry, spindown, binary, dispersion


## Part 2: MetaPulsar is an Enterprise Pulsar

The resulting MetaPulsar is a fully functional Enterprise pulsar with all standard attributes. It combines data from multiple PTAs into a single unified object.

### Key Features
- **Combined dataset**: All observations combined
- **Merged Astrophysics**: Only a single Astrophysical model
- **PTA-Specific Parameters**: Detector-specific parameters retain PTA suffixes
- **Full Enterprise Compatibility**: Works with all Enterprise analysis tools


In [4]:
# The resulting MetaPulsar is an Enterprise pulsar with all standard attributes
print("MetaPulsar Enterprise attributes:")
print(f"  Name: {metapulsar.name}")
print(f"  Number of pulsars: {len(metapulsar._pulsars)}")
print(f"  PTA names: {list(metapulsar._pulsars.keys())}")
print(f"  Combination strategy: {metapulsar.combination_strategy}")
print(f"  Model components merged: {metapulsar.combine_components}")

# Show some basic Enterprise pulsar attributes
print("\nEnterprise pulsar attributes:")
print(f"  Number of TOAs: {len(metapulsar.toas)}")
print(
    f"  Frequency range: {metapulsar.freqs.min():.2f} - {metapulsar.freqs.max():.2f} MHz"
)
print(f"  Time span: {(metapulsar.toas.max() - metapulsar.toas.min())/86400.0:.1f} days")

print(
    "\nThe MetaPulsar combines data from multiple PTAs into a single Enterprise pulsar."
)
print(
    f"Merged parameters inherit values from the reference PTA ({list(pulsar_data.keys())[0]})."
)
print("PTA-specific parameters retain their original PTA-specific values.")


MetaPulsar Enterprise attributes:
  Name: J0613-0200
  Number of pulsars: 3
  PTA names: ['epta_dr2', 'nanograv_9y', 'ppta_dr2']
  Combination strategy: consistent
  Model components merged: ['astrometry', 'spindown', 'binary', 'dispersion']

Enterprise pulsar attributes:
  Number of TOAs: 9201
  Frequency range: 323.72 - 3102.14 MHz
  Time span: 5863.9 days

The MetaPulsar combines data from multiple PTAs into a single Enterprise pulsar.
Merged parameters inherit values from the reference PTA (epta_dr2).
PTA-specific parameters retain their original PTA-specific values.


### Parameter Naming Conventions

MetaPulsar uses a clear naming convention to distinguish between merged and PTA-specific parameters:

- **Merged parameters**: No suffix, inherit reference PTA values
- **PTA-specific parameters**: Retain PTA suffix (e.g., `_nanograv_9y`, `_epta_dr2`)

This allows you to easily identify which parameters are shared across PTAs and which are specific to individual PTAs.


In [5]:
# Demonstrate parameter naming conventions
print("Parameter naming conventions:")
print("Merged parameters (no suffix):")
fitparams = metapulsar.fitpars
# Get PTA names from our data structure
pta_names = list(pulsar_data.keys())
pta_suffixes = [f"_{pta}" for pta in pta_names]

merged_params = [
    p for p in fitparams if not any(suffix in p for suffix in pta_suffixes)
]
for param in merged_params[:5]:  # Show first 5 merged parameters
    print(f"  {param}")

print("\nPTA-specific parameters (retain PTA suffix):")
pta_specific_params = [
    p for p in fitparams if any(suffix in p for suffix in pta_suffixes)
]
for param in pta_specific_params[:5]:  # Show first 5 PTA-specific parameters
    print(f"  {param}")

print("\nThis naming convention allows you to distinguish between:")
print("  - Merged parameters: Inherit reference PTA values, no suffix")
print("  - PTA-specific parameters: Retain original values, keep PTA suffix")


Parameter naming conventions:
Merged parameters (no suffix):
  PX
  RAJ
  DECJ
  PMRA
  PMDEC

PTA-specific parameters (retain PTA suffix):
  JUMP1_nanograv_9y
  FD1_nanograv_9y
  FD2_nanograv_9y
  Offset_nanograv_9y
  JUMP1_epta_dr2

This naming convention allows you to distinguish between:
  - Merged parameters: Inherit reference PTA values, no suffix
  - PTA-specific parameters: Retain original values, keep PTA suffix


## Part 3: Automated Multi-Pulsar Processing

For processing multiple pulsars, manually creating data structures becomes cumbersome. MetaPulsar provides utility functions based on regex pattern matching for automation.

### Automated Workflow
1. **Discover layouts**: Automatically detect data release directory structures
2. **Combine layouts**: Merge discovered layouts with predefined patterns
3. **Discover files**: Find all pulsar files using pattern matching
4. **Create MetaPulsars**: Process discovered pulsars automatically (limited to 3 for performance)

### Reference PTA Selection Strategies
- **Auto-selection**: Choose PTA with longest timespan per pulsar
- **Global reference**: Use same PTA as reference for all pulsars
- **Manual**: Reorder PTAs for specific pulsars

**Note**: For demonstration purposes, we limit processing to the first 3 pulsars to keep execution time reasonable.


### Directory Layout Discovery

In [6]:
# Directory layout is different for each PTA data release
# We provide a regexp discovery engine that helps. It's not perfect,
# but it works for the PTA data releases we have seen so far.

epta_layout = discover_layout('../../data/ipta-dr2/EPTA_v2.2')
ppta_layout = discover_layout('../../data/ipta-dr2/PPTA_dr1dr2')
nanograv_layout = discover_layout('../../data/ipta-dr2/NANOGrav_9y')

# Combine layouts with:
combined_layout = combine_layouts(epta_layout, ppta_layout, nanograv_layout)

Discovered layout in ../../data/ipta-dr2/EPTA_v2.2:
  - base_dir = '../../data/ipta-dr2/EPTA_v2.2'
  - par_pattern = '([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?)/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.par'
  - tim_pattern = '([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?)/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.tim'
  - timing_package = 'tempo2'
  - description = 'Auto-discovered PTA from ../../data/ipta-dr2/EPTA_v2.2'
Discovered layout in ../../data/ipta-dr2/PPTA_dr1dr2:
  - base_dir = '../../data/ipta-dr2/PPTA_dr1dr2'
  - par_pattern = 'par/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.par'
  - tim_pattern = 'tim/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.tim'
  - timing_package = 'tempo2'
  - description = 'Auto-discovered PTA from ../../data/ipta-dr2/PPTA_dr1dr2'
Discovered layout in ../../data/ipta-dr2/NANOGrav_9y:
  - base_dir = '../../data/ipta-dr2/NANOGrav_9y'
  - par_pattern = 'par/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.par'
  - tim_pattern = 'tim/([BJ]\\d{4}[+-]\\d{2,4}[A-Z]?).*\\.tim'
  - timing_package = 'pint'
  - description = 'Auto

In [16]:
# The pre-defined PTA data release regexp patterns are:
from metapulsar import PTA_DATA_RELEASES
PTA_DATA_RELEASES.keys(), PTA_DATA_RELEASES['epta_dr2']

(dict_keys(['epta_dr1_v2_2', 'epta_dr2', 'ppta_dr2', 'nanograv_9y', 'inpta_dr1', 'mpta_dr1', 'nanograv_12y', 'nanograv_15y']),
 {'base_dir': 'data/IPTA-DR3/EPTA_DR2/',
  'par_pattern': '([BJ]\\d{4}[+-]\\d{2,4})/\\1\\.par',
  'tim_pattern': '([BJ]\\d{4}[+-]\\d{2,4})/\\1_all\\.tim',
  'timing_package': 'tempo2',
  'description': 'EPTA Data Release 2'})

#### File discovery
Those regexp patterns can be fed to the file discovery service. There are some default regexp expressions as well, but it's unlikely that they match the exact structure you have in your data release

In [7]:
file_data = discover_files(combined_layout)

Found:
  - discovered_EPTA_v2.2: 42 pulsars
  - discovered_NANOGrav_9y: 37 pulsars
  - discovered_PPTA_dr1dr2: 18 pulsars


# Filtering for pulsars

The file_data is just a dictionary with file path information. The pulsars are _not_ yet matched between PTAs. This is not done by name, but by coordinate. We provide some utility functions to see which pulsars are in this file structure

In [8]:
from metapulsar import get_pulsar_names_from_file_data, filter_file_data_by_pulsars

# Do coordinate-based pulsar matching between PTAs
pulsar_names = get_pulsar_names_from_file_data(file_data)

len(pulsar_names), pulsar_names[:5]

(66, ['J0030+0451', 'J0034-0534', 'J0218+4232', 'J0610-2100', 'J0613-0200'])

In [13]:
# We filter to only include 3 pulsars
# In this step we can choose both the 'B' and the 'J' name! The names are resolved automagically by coordinates. Honeybadger don't care.
pulsar_selection = ['B1855+09', 'J1939+2135', 'J0030+0451']

filtered_data = filter_file_data_by_pulsars(file_data, pulsar_selection)
filtered_data

{'discovered_EPTA_v2.2': [{'par': PosixPath('/workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943.par'),
   'tim': PosixPath('/workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim'),
   'timing_package': 'tempo2',
   'timespan_days': 6322.58259303341,
   'par_content': 'PSRJ           J1857+0943\nRAJ             18:57:36.3906094         1  0.00000377562506537700   \nDECJ           +09:43:17.20701           1  0.00011409033390585553   \nF0             186.49407837798747865     1  0.00000000000242696879   \nF1             -6.2050914708832126377e-16 1  3.358585861050660548e-21 \nPEPOCH         55000                       \nPOSEPOCH       55000                       \nDMEPOCH        55000                       \nDM             13.307305292320601406     1  0.00416837516446324791   \nDM1            0.0013916212021063438124  1  0.00019759875708597415   \nDM2\t\t0.0\t\t\t1\nPMRA           -2.63901120670936

# Part 2: Create MetaPulsars for all discovered pulsars

We can now create MetaPulsars for all discovered pulsars.
We can either auto-select a reference PTA for each pulsar, or use a specific PTA for all pulsars.

### Reference PTA
The reference PTA is the PTA for which the par-file values are taken as 'reference values'. While these parameters are marginalized over, we do need to gauge them and lock them together. This is done by choosing one of the PTAs, and using those values in the par files. This choice is usually not important for the outcome, especially for millisecond pulsars. However, it's wise to choose the most accurate model for it. Usually, the longest PTA dataset is a good bet.


In [11]:
metapulsars = factory.create_all_metapulsars(filtered_data, reference_pta=None) #='discovered_NANOGrav_9y')
# The default options are:
# combination_strategy="consistent",
# combine_components=["astrometry", "spindown", "binary", "dispersion"],
# add_dm_derivatives=True,

# The reference_pta=None option means that the reference PTA will be auto-selected for each pulsar based on observation timespans.

2025-10-09 19:44:10.278 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: DD.
2025-10-09 19:44:10.285 | WARNING  | pint.models.model_builder:__call__:229 - UNITS is not specified. Assuming TDB...
2025-10-09 19:44:10.424 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: DD.
2025-10-09 19:44:10.439 | WARNING  | pint.models.timing_model:validate:426 - PINT does not support 'UNITS TCB' internally. Reading this par file nevertheless because the `allow_tcb` option was given. This `TimingModel` object should not be used for anything except converting to TDB.
2025-10-09 19:44:10.440 | WARNING  | pint.models.tcb_conversion:convert_tcb_tdb:133 - Converting this timing model from TCB to TDB. Please note that the TCB to TDB conversion is only approximate and the resulting timing model should be re-fit to get reliable results.


/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:406: UserWarning: PINT only supports 'T2CMETHOD IAU2000B'
  warn("PINT only supports 'T2CMETHOD IAU2000B'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:400: UserWarning: PINT does not support 'DILATEFREQ Y'
  warn("PINT does not support 'DILATEFREQ Y'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:403: UserWarning: PINT only supports 'TIMEEPH FB90'
  warn("PINT only supports 'TIMEEPH FB90'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:432: UserWarning: START cannot be unfrozen... Setting START.frozen to True
  warn("START cannot be unfrozen... Setting START.frozen to True")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_mod

2025-10-09 19:44:10.512 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: DD.
2025-10-09 19:44:10.653 | WARNING  | pint.models.model_builder:choose_binary_model:627 - Found T2 binary model. Gracefully converting T2 to: DD.
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J1857+0943/tims/JBO.DFB.1400.tim


/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'TNSUBTRACTPOLY 1'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'DM_SERIES TAYLOR'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)


Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J1857+0943/tims/JBO.DFB.1520.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J1857+0943/tims/EFF.EBPP.1360.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J1857+0943/tims/EFF.EBPP.1410.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J1857+0943/tims/EFF.EBPP.2639.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J1857+0943/J1857+0943_all.tim
Rel path = /workspaces/metap

...

/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:406: UserWarning: PINT only supports 'T2CMETHOD IAU2000B'
  warn("PINT only supports 'T2CMETHOD IAU2000B'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:400: UserWarning: PINT does not support 'DILATEFREQ Y'
  warn("PINT does not support 'DILATEFREQ Y'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:403: UserWarning: PINT only supports 'TIMEEPH FB90'
  warn("PINT only supports 'TIMEEPH FB90'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'TNSUBTRACTPOLY 1'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:2

Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J0030+0451/J0030+0451_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0030+0451/tims/EFF.EBPP.1360.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J0030+0451/J0030+0451_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0030+0451/tims/EFF.EBPP.1410.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J0030+0451/J0030+0451_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0030+0451/tims/EFF.EBPP.2639.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J0030+0451/J0030+0451_all.tim
Rel path = /workspaces/metapulsar/data/ipta-dr2/EPTA_v2.2/J0030+0451/tims/NRT.BON.1400.tim
Current filename = /workspaces/metapulsar/examples/notebooks/../../data/ipta-dr2/EPTA_v2.2/J0030+0451/J0030+0451_all.tim
Rel path = /workspaces/metap

/opt/venvs/pta/lib/python3.10/site-packages/pint/observatory/clock_file.py:183: UserWarning: Data points out of range in clock file 'tai2tt_bipm2011.clk'
  warn(msg)




Results for PSR J0030+0451


RMS pre-fit residual = 0.000 (us), RMS post-fit residual = 7.094 (us)
Fit Chisq = 0	Chisqr/nfree = 0.00/0 = nan	pre/post = 0
Number of fit parameters: 0
Number of points in fit = 0
Offset: 0 1 offset_e*sqrt(n) = 0 n = 0


PARAMETER       Pre-fit                   Post-fit                  Uncertainty   Difference   Fit
---------------------------------------------------------------------------------------------------
RAJ (rad)       0.132894341420816         0.132894341420816         0             0             Y
RAJ (hms)       00:30:27.4283723           00:30:27.4283723         0             0            
DECJ (rad)      0.0848409743203409        0.0848409743203409        0             0             Y
DECJ (dms)      +04:51:39.70713           +04:51:39.70713           0             0            
F0 (s^-1)       205.530699125258          205.530699125258          0             0             Y
F1 (s^-2)       -4.29716676975486e-16     -4.29716676975486e-1

/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:400: UserWarning: PINT does not support 'DILATEFREQ Y'
  warn("PINT does not support 'DILATEFREQ Y'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/timing_model.py:403: UserWarning: PINT only supports 'TIMEEPH FB90'
  warn("PINT only supports 'TIMEEPH FB90'")
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'EPHVER 5'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'DMASSPLANET1 0'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/models/model_builder.py:225: UserWarning: Unrecognized parfile line 'DMASSPLANET2 0'
  warnings.warn(f"Unrecognized parfile line '{p_line}'", UserWarning)
/opt/venvs/pta/lib/python3.10/site-packages/pint/mod

In [12]:
# Now we have a full list of MetaPulsars
metapulsars

{'J1857+0943': <metapulsar.metapulsar.MetaPulsar at 0xfffeebeda290>,
 'J0030+0451': <metapulsar.metapulsar.MetaPulsar at 0xfffee920ecb0>,
 'J1939+2135': <metapulsar.metapulsar.MetaPulsar at 0xfffee6c32c50>}